In [1]:
import random
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
x = np.load("./data/balanced/input_data.npy")
labels = np.load("./data/balanced/label_data.npy", allow_pickle=True)

In [3]:
x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()

In [4]:
unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels)
targets = targets.long()

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [5]:
# x_min = inputs.min()
# x_max = inputs.max()
# print((x_min, x_max))

# inputs = 100 * (inputs - x_min) / (x_max - x_min) -1

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=3, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 6)

    def forward(self, x):
        conv1 = self.pool(F.tanh(self.conv1(x)))  # torch.Size([13042, 16, 24, 24])
        conv2 = self.pool(F.tanh(self.conv2(conv1)))  # torch.Size([13042, 32, 12, 12])
        conv3 = self.pool(F.tanh(self.conv3(conv2)))  # torch.Size([13042, 64, 6, 6])

        flatten = conv3.view(-1, 64 * 6 * 6)

        fc1 = F.tanh(self.fc1(flatten))
        fc2 = F.tanh(self.fc2(fc1))

        return fc2




In [8]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0006)

warmup_steps = 5
epochs = 1000

schedular = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

model = model.to(device=device)
inputs = inputs.to(device=device)
targets = targets.to(device=device)

for epoch in range(epochs):
    # if epoch > and epoch > warmup_steps:
    #     warmup_lr = 0.001 * (epoch + 1) / warmup_steps
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = warmup_lr
    # else:
    #     schedular.step()


    optimizer.zero_grad()
    outputs = model.forward(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {loss.item()}, Learning Rate: {current_lr}")

In [17]:
x = np.load("./data/balanced/test_data.npy")
labels = np.load("./data/balanced/test_label_data.npy", allow_pickle=True)

x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()
inputs = inputs.to(device)

unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels)
targets = targets.long()
targets = targets.to(device=device)

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [18]:
y_pred = model(inputs)

In [11]:
probabilities = F.softmax(y_pred, dim=1)

# Get the predicted class with the highest probability
predicted_class = torch.argmax(probabilities, dim=1)

In [19]:
predicted_class = predicted_class.to(device=device)
print(targets)

tensor([3, 5, 1,  ..., 4, 5, 3], device='cuda:0')


In [21]:
accuracy_metric  = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device=device)
accuracy = accuracy_metric(predicted_class, targets)

In [24]:
accuracy

tensor(0.4440, device='cuda:0')